# Cartopy tutorial: Transforming raster to vector

Exercise: Extract contours from a raster, save to GeoJSON.

*Exercise:* Pull out the route from the wikipedia image.

*Exercise:* Project the geometry to PC, so that we can save it to GeoJSON.